# subprocess.Popen

In [1]:
%%writefile date1.py

import subprocess

subprocess.Popen('date')

Writing date1.py


In [2]:
!python3.4 date1.py

Sun May 17 01:25:15 YEKT 2015


In [3]:
%%writefile date2.py

import subprocess

subprocess.Popen(['date', '+%H:%M:%S'])

Writing date2.py


In [4]:
!python3.4 date2.py

01:25:17


In [5]:
%%writefile date3.py

import subprocess

subprocess.Popen('date +%H:%M:%S', shell=True)

Writing date3.py


In [6]:
!python3.4 date3.py

01:25:20


In [7]:
%%writefile date4.py

import subprocess

subprocess.Popen('date', stdout=subprocess.DEVNULL)

Writing date4.py


In [8]:
!python3.4 date4.py

In [9]:
import subprocess

p = subprocess.Popen('cat', stdin=subprocess.PIPE, stdout=subprocess.PIPE)
p.stdin.write(b'Hello!\n')
p.stdin.close()
print(p.poll())
print(p.stdout.read())
print(p.poll())

None
b'Hello!\n'
0


# fork

In [1]:
%%writefile fork.py

import os

pid = os.fork()
if pid:
    # parent
    for _ in range(200):
        print('hello from parent')
else:
    # child
    for _ in range(200):
        print('hello from child')

Writing fork.py


In [2]:
!python3.4 fork.py

hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from parent
hello from

# threading

## def

In [1]:
%%writefile threads1.py

import threading

def action():
    for _ in range(10):
        print('hello from', threading.current_thread().name)

for _ in range(10):
    thread = threading.Thread(target=action)
    thread.start()

Writing threads1.py


In [2]:
!python3.4 threads1.py

hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-1
hello from Thread-3
hello from Thread-2
hello from Thread-1
hello from Thread-5
hello from Thread-4
hello from Thread-7
hello from Thread-1
hello from Thread-3
hello from Thread-2
hello from Thread-4
hello from Thread-4
hello from Thread-8
hello from Thread-5
hello from Thread-9
hello from Thread-6
hello from Thread-1
hello from Thread-10
hello from Thread-3
hello from Thread-2
hello from Thread-7
hello from Thread-4
hello from Thread-8
hello from Thread-3
hello from Thread-9
hello from Thread-7
hello from Thread-4
hello from Thread-8
hello from Thread-3
hello from Thread-9
hello from Thread-7
hello from Thread-4
hello from Thread-8
hello from Thread-3
hello from Thread-9
hello from Thread-7
hello from Thread-4
hello from Thread-4

## class

In [1]:
%%writefile threads2.py

import threading

class Worker(threading.Thread):
    def run(self):
        for _ in range(10):
            print('hello from', threading.current_thread().name)

for _ in range(10):
    thread = Worker()
    thread.start()

Writing threads2.py


In [2]:
!python3.4 threads2.py

hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-1
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-2
hello from Thread-3
hello from Thread-3
hello from Thread-3
hello from Thread-3
hello from Thread-3
hello from Thread-3
hello from Thread-3
hello from Thread-3
hello from Thread-3
hello from Thread-3
hello from Thread-4
hello from Thread-4
hello from Thread-4
hello from Thread-4
hello from Thread-4
hello from Thread-4
hello from Thread-4
hello from Thread-4
hello from Thread-4
hello from Thread-4
hello from Thread-5
hello from Thread-5
hello from Thread-5
hello from Thread-5
hello from Thread-5
hello from Thread-5
hello from Thread-5
hello from Thread-5
hello from Thread-5
hello from Thread-5


## Неатомарный инкремент в препарированном виде

In [1]:
import threading
import time

counter = 0

def inc():
    global counter
    v = counter
    time.sleep(0.001)
    counter = v + 1

pool = []
for _ in range(1000):
    th = threading.Thread(target=inc)
    th.start()
    pool.append(th)

for th in pool:
    th.join()

print(counter)

152


## Неатомарный инкремент

In [1]:
import threading

counter = 0

def inc():
    global counter
    for _ in range(100000):
        counter += 1

pool = []
for _ in range(10):
    th = threading.Thread(target=inc)
    th.start()
    pool.append(th)

for th in pool:
    th.join()

print(counter)

903724


## Синхронизация потоков при помощи блокировки

In [1]:
import threading
import time

counter = 0
lock = threading.Lock()

def inc():
    global counter
    lock.acquire()
    v = counter
    time.sleep(0.001)
    counter = v + 1
    lock.release()

pool = []
for _ in range(1000):
    th = threading.Thread(target=inc)
    th.start()
    pool.append(th)

for th in pool:
    th.join()

print(counter)

1000


## Использование with вместе с lock

In [1]:
import threading
import time

counter = 0
lock = threading.Lock()

def inc():
    global counter
    with lock:
        v = counter
        time.sleep(0.001)
        counter = v + 1

pool = []
for _ in range(1000):
    th = threading.Thread(target=inc)
    th.start()
    pool.append(th)

for th in pool:
    th.join()

print(counter)

1000


## Взаимные блокировки, deadlock

In [3]:
import threading
import time

lock1 = threading.Lock()
lock2 = threading.Lock()

def good():
    with lock1:
        time.sleep(0.001)
        if lock2.acquire(timeout=1):
            print('good')
            lock2.release()
        else:
            print('locking failed :(')

def bad():
    with lock2:
        if lock1.acquire(timeout=1):
            print('bad')
            lock1.release()
        else:
            print('locking failed :(')

threading.Thread(target=good).start()
threading.Thread(target=bad).start()

locking failed :(
locking failed :(
